In [22]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
import sys

In [17]:
#one hot
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

# imput_dim 4 => output_dim 2 
#cell = nn.RNN(input_size=4, hidden_size=2, batch_first=True)

# num_layers * num_directions, batch, hidden_size 
hidden = Variable(torch.randn(1,1,2)) #Tensor size 1x1x2
#print(hidden)

inputs = Variable(torch.Tensor([h,e,l,l,o])) #floatTensor size 5x4
#print(inputs)

for one in inputs:
    one = one.view(1, 1, -1)
    #print(one)
    out, hidden = cell(one, hidden)
    #print("one input size : ", one.size(), "out size : ", out.size())

inputs = inputs.view(1, 5, -1)
#print(inputs)
out, hidden = cell(inputs, hidden)
#print("seq input size : ", inputs.size(), "out size : ", out.size())

hidden = Variable(torch.randn(1, 3, 2))
print(hidden)

# 3 batches hello eolll lleel
inputs = Variable(torch.Tensor([[h, e, l, l, o],
                               [e, o, l, l, l],
                               [l, l, e, e, l]]))
#print(inputs) # size 3x5x4
cell = nn.RNN(input_size=4, hidden_size=2)
inputs = inputs.transpose(dim0=0, dim1=1) #size 5x3x4
#print(inputs)

# one cell RNN input_dim 4 output_dim 2
cell = nn.RNN(input_size=4, hidden_size=2, batch_first=True)

inputs = inputs.transpose(dim0=0, dim1=1)

out, hidden = cell(inputs, hidden)
print('batch input size : ',inputs.size(), 'out size : ', out.size())


Variable containing:
(0 ,.,.) = 
 -0.8735  1.9095
 -0.2230 -1.1994
  0.8567 -0.7846
[torch.FloatTensor of size 1x3x2]

batch input size :  torch.Size([3, 5, 4]) out size :  torch.Size([3, 5, 2])


In [27]:
torch.manual_seed(777)

idx2char = ['h', 'i', 'e', 'l', 'o']

x_data = [0, 1, 0, 2, 3, 3] #hihell
one_hot_lookup = [[1, 0, 0, 0, 0],
                 [0, 1, 0, 0, 0],
                 [0, 0, 1, 0, 0],
                 [0, 0, 0, 1, 0],
                 [0, 0, 0, 0, 1]]

y_data = [1, 0, 2, 3, 3, 4] #ihello
x_one_hot = [one_hot_lookup[x] for x in x_data]
#print(x_one_hot)

inputs = Variable(torch.Tensor(x_one_hot))
labels = Variable(torch.LongTensor(y_data))

num_classes = 5
input_size = 5 # one_hot_size
hidden_size = 5 # output from RNN. 5 to directly predict one-hot
batch_size = 1 # one sentence
sequence_length = 1 # one by one
num_layers = 1 # one-layer rnn

class Model(nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        self.rnn = nn.RNN(input_size=input_size,
                         hidden_size=hidden_size, batch_first=True)
        
    def forward(self, hidden, x):
        # reshape input (batch first)
        x = x.view(batch_size, sequence_length, input_size)
        #print(x)
        out, hidden = self.rnn(x, hidden)
        return hidden, out.view(-1, num_classes)
    
    def init_hidden(self):
        
        
        return Variable(torch.zeros(num_layers, batch_size, hidden_size))

model = Model() # RNN(5, 5)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

for epoch in range(100):
    optimizer.zero_grad()
    loss = 0
    hidden = model.init_hidden()
    
    sys.stdout.write("predicted string: ")
    for input, label in zip(inputs, labels):
        #print(input.size(), label.size())
        hidden, output = model(hidden, input)
        val, idx = output.max(1)
        sys.stdout.write(idx2char[idx.data[0]])
        loss += criterion(output, label)
    

    print(", epoch: %d, loss: %1.3f"% (epoch + 1, loss.data[0]))
    
    loss.backward()
    optimizer.step()
    
print("Learning finished!")
    
    
        
    

Model(
  (rnn): RNN(5, 5, batch_first=True)
)
predicted string: llllll, epoch: 1, loss: 10.155
predicted string: llllll, epoch: 2, loss: 9.137
predicted string: llllll, epoch: 3, loss: 8.355
predicted string: llllll, epoch: 4, loss: 7.577
predicted string: llllll, epoch: 5, loss: 6.876
predicted string: lhelll, epoch: 6, loss: 6.327
predicted string: ihelll, epoch: 7, loss: 6.014
predicted string: ihelll, epoch: 8, loss: 5.787
predicted string: ihelll, epoch: 9, loss: 5.477
predicted string: ihelll, epoch: 10, loss: 5.274
predicted string: ihelll, epoch: 11, loss: 5.041
predicted string: ihello, epoch: 12, loss: 4.827
predicted string: ihello, epoch: 13, loss: 4.676
predicted string: ihello, epoch: 14, loss: 4.550
predicted string: ihello, epoch: 15, loss: 4.430
predicted string: ihello, epoch: 16, loss: 4.305
predicted string: ihello, epoch: 17, loss: 4.164
predicted string: ihelll, epoch: 18, loss: 4.003
predicted string: ihelll, epoch: 19, loss: 3.860
predicted string: ihelll, epoch

In [60]:
import numpy as np
data = 10000
sen = np.random.rand(5)
seq = np.random.rand(100)
sent = []
for i in range(5):
    sent.append(seq)
#print(sent)
zero_padding = np.zeros((data, 20), dtype=np.int32)

zero_padding[0,:5] = np.array(sen)
print(zero_padding)
print([sum(x) for x in sent])


[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
[48.589802800382174, 48.589802800382174, 48.589802800382174, 48.589802800382174, 48.589802800382174]


In [59]:
l=[[1,2],[3,4]]
#[x+y for x,y in l]
[sum(x) for x in l]

[3, 7]